# Robotic Bin-Picking System Demo

This notebook demonstrates the use of the bin-picking system for object detection,
pose estimation, and gripper selection.

## Overview

The system consists of four main modules:
1. **clustering.py** - Point cloud preprocessing and object segmentation
2. **pose_estimation.py** - 6D pose estimation using PCA
3. **grasp_planner.py** - Gripper selection with collision avoidance
4. **action_generator.py** - Complete detection and planning pipeline

## Setup

First, install required dependencies and import necessary modules.

In [ ]:
# Import required libraries
import sys
import os
import numpy as np
import open3d as o3d
import cv2
import yaml
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Add parent directory to path
sys.path.insert(0, os.path.abspath('..'))

# Import bin-picking modules
from src import BinPickingSystem, DEFAULT_GRIPPER_SPECS

print("All imports successful!")

## Load Data

Load RGB image, point cloud, and camera parameters.

**Note:** Update the data paths below to point to your dataset location.

In [ ]:
# Define data file paths
DATA_DIR = "../data"  # Update this to your data directory

FILES = {
    'rgb_image': os.path.join(DATA_DIR, 'Color_3840x2160.png'),
    'point_cloud': os.path.join(DATA_DIR, 'pcd.ply'),
    'rgb_intrinsic': os.path.join(DATA_DIR, 'rgb_intrinsic.yaml'),
    'depth_intrinsic': os.path.join(DATA_DIR, 'depth_intrinsic.yaml'),
    'rgb_distortion': os.path.join(DATA_DIR, 'rgb_distortion.yaml')
}

# Load camera parameters
def load_camera_params(intrinsic_path, distortion_path=None):
    with open(intrinsic_path, 'r') as f:
        intrinsics = yaml.safe_load(f)
    
    camera_params = {
        'fx': float(intrinsics['fx']),
        'fy': float(intrinsics['fy']),
        'cx': float(intrinsics['cx']),
        'cy': float(intrinsics['cy']),
        'width': int(intrinsics['width']),
        'height': int(intrinsics['height'])
    }
    
    camera_params['K'] = np.array([
        [camera_params['fx'], 0, camera_params['cx']],
        [0, camera_params['fy'], camera_params['cy']],
        [0, 0, 1]
    ], dtype=np.float64)
    
    return camera_params

# Load RGB image
rgb_image = cv2.imread(FILES['rgb_image'])
rgb_image = cv2.cvtColor(rgb_image, cv2.COLOR_BGR2RGB)

# Load point cloud
pcd = o3d.io.read_point_cloud(FILES['point_cloud'])

# Load camera parameters
camera_params = load_camera_params(FILES['rgb_intrinsic'], FILES['rgb_distortion'])

print(f"RGB Image Shape: {rgb_image.shape}")
print(f"Point Cloud Points: {len(pcd.points):,}")
print(f"Camera Resolution: {camera_params['width']}x{camera_params['height']}")

## Initialize Bin-Picking System

Create an instance of the bin-picking system with default parameters.

In [ ]:
# Initialize system
system = BinPickingSystem(
    voxel_size=2.0,
    dbscan_eps=40.0,
    dbscan_min_points=100,
    gripper_specs=DEFAULT_GRIPPER_SPECS
)

print("Bin-picking system initialized!")
print(f"\nGripper Specifications:")
print(f"  Finger Gripper Max Width: {DEFAULT_GRIPPER_SPECS['finger_gripper']['max_width']} mm")
print(f"  Suction Cup Diameter: {DEFAULT_GRIPPER_SPECS['suction_gripper']['cup_diameter']} mm")

## Process Scene

Run the complete pipeline: object detection, pose estimation, and grasp planning.

In [ ]:
# Process the scene
detected_objects, grasp_plans, bin_dims = system.process_scene(
    rgb_image=rgb_image,
    point_cloud=pcd,
    camera_params=camera_params,
    debug=True
)

print(f"\n{'='*80}")
print(f"DETECTION RESULTS")
print(f"{'='*80}")
print(f"\nDetected {len(detected_objects)} objects")
print(f"\nBin Dimensions:")
print(f"  Width: {bin_dims['width']:.1f} mm")
print(f"  Depth: {bin_dims['depth']:.1f} mm")
print(f"  Height: {bin_dims['height']:.1f} mm")

## Analyze Results

View statistics about the gripper selections.

In [ ]:
# Analyze gripper selections
analysis = system.analyze_results(grasp_plans)

print(f"\n{'='*80}")
print(f"GRIPPER SELECTION ANALYSIS")
print(f"{'='*80}")
print(f"\nTotal Objects: {analysis['total_objects']}")
print(f"  Finger Gripper: {analysis['finger_count']} ({analysis['finger_percentage']:.1f}%)")
print(f"  Suction Gripper: {analysis['suction_count']} ({analysis['suction_percentage']:.1f}%)")
print(f"\nConfidence Statistics:")
if analysis['finger_count'] > 0:
    print(f"  Finger Gripper Mean: {analysis['finger_mean_conf']:.2%}")
if analysis['suction_count'] > 0:
    print(f"  Suction Gripper Mean: {analysis['suction_mean_conf']:.2%}")
print(f"\nHigh Confidence Picks (>70%): {analysis['high_confidence_count']} ({analysis['high_confidence_percentage']:.1f}%)")
print(f"Objects with Collision Issues: {analysis['collision_count']}")

## Display Top Grasp Candidates

Show the top 5 objects sorted by grasp confidence.

In [ ]:
# Display top 5 grasp plans
print(f"\n{'='*80}")
print(f"TOP 5 GRASP CANDIDATES (sorted by confidence)")
print(f"{'='*80}")

for i, (obj, decision) in enumerate(grasp_plans[:5]):
    print(f"\n[{i+1}] Object ID: {obj['id']}")
    print(f"  Position: [{obj['position'][0]:.1f}, {obj['position'][1]:.1f}, {obj['position'][2]:.1f}] mm")
    print(f"  Size: {obj['bounding_box']['width']:.1f} x {obj['bounding_box']['height']:.1f} x {obj['bounding_box']['depth']:.1f} mm")
    print(f"  Surface: {obj['surface_quality']}")
    print(f"  Detection Confidence: {obj['confidence']:.2%}")
    print(f"  \n  Gripper: {decision['gripper']}")
    print(f"  Grasp Confidence: {decision['confidence']:.2%}")
    print(f"  Reason: {decision['reason']}")
    print(f"  -" * 40)

## Visualize Results

Visualize the RGB image and detected objects.

In [ ]:
# Visualize RGB image
fig, ax = plt.subplots(1, 1, figsize=(12, 8))
ax.imshow(rgb_image)
ax.set_title(f'RGB Image - {len(detected_objects)} Objects Detected', fontsize=14, fontweight='bold')
ax.axis('off')
plt.tight_layout()
plt.show()

In [ ]:
# Visualize point cloud with detected objects
points = np.asarray(pcd.points)
colors = np.asarray(pcd.colors) if pcd.has_colors() else np.ones_like(points) * 0.5

fig = plt.figure(figsize=(14, 10))
ax = fig.add_subplot(111, projection='3d')

# Plot downsampled point cloud
subsample = 10
points_sub = points[::subsample]
colors_sub = colors[::subsample]

ax.scatter(points_sub[:, 0], points_sub[:, 1], points_sub[:, 2],
           c=colors_sub, s=0.5, alpha=0.3)

ax.set_xlabel('X (mm)', fontweight='bold')
ax.set_ylabel('Y (mm)', fontweight='bold')
ax.set_zlabel('Z (mm)', fontweight='bold')
ax.set_title(f'3D Point Cloud - {len(detected_objects)} Objects Detected',
             fontsize=14, fontweight='bold')
ax.view_init(elev=25, azim=45)

plt.tight_layout()
plt.show()

## Summary

This demo showed how to:
1. Load RGB-D data and camera parameters
2. Initialize the bin-picking system
3. Detect objects and estimate poses
4. Plan grasps with collision avoidance
5. Analyze and visualize results

For more details, see the individual module documentation:
- `src/clustering.py` - Point cloud processing
- `src/pose_estimation.py` - Pose estimation algorithms
- `src/grasp_planner.py` - Gripper selection logic
- `src/action_generator.py` - Complete pipeline integration